# The Actuator Line Function
This first block of code is almost completely unchanged from the actual WindSE implementation.  In the latest version, it looks for a variable `mpi_u_fluid` assembled outside the function rather than probing the velocity field given by u_local.  However, since those changes would require extending this demo even further, the `mpi_u_fluid` variable goes unused and the velocity is probed directly.

![Turbine Force](turbine_force_example.png)


In [1]:
from fenics import *
import numpy as np
import matplotlib.pyplot as plt
import scipy.interpolate as interp
import glob

def UpdateActuatorLineForce(problem, u_local, simTime_id, dt, turb_i, mpi_u_fluid, dfd=None, verbose=False):

    simTime = problem.simTime_list[simTime_id]

    if verbose:
        print("Current Optimization Time: "+repr(simTime)+", Turbine #"+repr(turb_i))
        sys.stdout.flush()

    def rot_x(theta):
        Rx = np.array([[1, 0, 0],
                       [0, np.cos(theta), -np.sin(theta)],
                       [0, np.sin(theta), np.cos(theta)]])

        return Rx

    def rot_y(theta):
        Ry = np.array([[np.cos(theta), 0, np.sin(theta)],
                       [0, 1, 0],
                       [-np.sin(theta), 0, np.cos(theta)]])
        
        return Ry

    def rot_z(theta):
        Rz = np.array([[np.cos(theta), -np.sin(theta), 0],
                       [np.sin(theta), np.cos(theta), 0],
                       [0, 0, 1]])
        
        return Rz

    def write_lift_and_drag(fn, simTime, theta, forceVec):
        fp = open('./output/%s/nodal_%s.csv' % (problem.params.name, fn), 'a')
        fp.write('%.5e, %.5e, ' % (simTime, theta))

        N = np.size(forceVec)

        for j in range(N):
            if j < N-1:
                fp.write('%.5e, ' % (forceVec[j]))
            else:
                fp.write('%.5e\n' % (forceVec[j]))
                
        fp.close()

    def save_derivative_file(folder,filename, deriv_array):

        dolfin_function = Function(problem.fs.V)

        fp = File(folder+'%s.pvd' % (filename))

        for k in range(problem.num_blade_segments):
            vec = np.copy(deriv_array[:, k])

            # Zero-out all very small values (<1e-12)
            vec[np.abs(vec) < 1e-12] = 0.0

            # Set the values of the dolfin function being saved
            dolfin_function.vector()[:] = vec

            # Rename the function for ParaView naming consistency
            dolfin_function.rename(filename, filename)

            # Save the function            
            fp << (dolfin_function, k)

    def build_lift_and_drag(problem, u_rel, blade_unit_vec, rdim, twist, c):

        def get_angle_between_vectors(a, b, n):
            a_x_b = np.dot(np.cross(n, a), b)

            norm_a = np.sqrt(a[0]*a[0] + a[1]*a[1] + a[2]*a[2])
            norm_b = np.sqrt(b[0]*b[0] + b[1]*b[1] + b[2]*b[2])

            c1 = a_x_b/(norm_a*norm_b)
            c1 = np.clip(c1, -1.0, 1.0)
            aoa_1 = np.arcsin(c1)

            c2 = np.dot(a, b)/(norm_a*norm_b)
            c2 = np.clip(c2, -1.0, 1.0)
            aoa_2 = np.arccos(c2)
            
            if aoa_2 > np.pi/2.0:
                if aoa_1 < 0:
                    aoa_1 = -np.pi - aoa_1
                else:
                    aoa_1 = np.pi - aoa_1
            
            aoa_1_deg = aoa_1/np.pi*180.0
            
            return aoa_1


        # If this is the first time calling the function...
        # if problem.first_call_to_alm: # This breaks in parallel, use the version below

        if not hasattr(problem, 'interp_lift'):
            # build the lift-drag table interpolators
            rdim_all = np.linspace(0, rdim[-1], np.shape(problem.lift_table)[1])
            problem.interp_lift = interp.RectBivariateSpline(problem.interp_angles, rdim_all, problem.lift_table)
            problem.interp_drag = interp.RectBivariateSpline(problem.interp_angles, rdim_all, problem.drag_table)


        # Initialize the real cl and cd profiles
        real_cl = np.zeros(problem.num_blade_segments)
        real_cd = np.zeros(problem.num_blade_segments)

        # fp = open(problem.aoa_file, 'a')

        tip_loss = np.zeros(problem.num_blade_segments)

        for k in range(problem.num_blade_segments):
            # Get the relative wind velocity at this node
            wind_vec = u_rel[:, k]

            # Remove the component in the radial direction (along the blade span)
            wind_vec -= np.dot(wind_vec, blade_unit_vec[:, 1])*blade_unit_vec[:, 1]

            # aoa = get_angle_between_vectors(arg1, arg2, arg3)
            # arg1 = in-plane vector pointing opposite rotation (blade sweep direction)
            # arg2 = relative wind vector at node k, including blade rotation effects (wind direction)
            # arg3 = unit vector normal to plane of rotation, in this case, radially along span
            aoa = get_angle_between_vectors(-blade_unit_vec[:, 2], wind_vec, -blade_unit_vec[:, 1])

            # Compute tip-loss factor
            if rdim[k] < 1e-12:
                tip_loss[k] = 1.0
            else:
                loss_exponent = 3.0/2.0*(rdim[-1]-rdim[k])/(rdim[k]*np.sin(aoa))
                acos_arg = np.exp(-loss_exponent)
                acos_arg = np.clip(acos_arg, -1.0, 1.0)
                tip_loss[k] = 2.0/np.pi*np.arccos(acos_arg)

            # Remove the portion of the angle due to twist
            aoa -= twist[k]

            # Store the cl and cd by interpolating this (aoa, span) pair from the tables
            real_cl[k] = problem.interp_lift(aoa, rdim[k])
            real_cd[k] = problem.interp_drag(aoa, rdim[k])

            # Write the aoa to a file for future reference
            # fp.write('%.5f, ' % (aoa/np.pi*180.0))

        # fp.close()

        return real_cl, real_cd, tip_loss


    #================================================================
    # Get Mesh Properties
    #================================================================

    ndim = problem.dom.dim

    # Initialize a cumulative turbine force function (contains all turbines)
    tf = Function(problem.fs.V)
    tf.vector()[:] = 0.0

    # Initialize a cylindrical field function
    cyld = Function(problem.fs.V)

    #================================================================
    # Set Turbine and Fluid Properties
    #================================================================

    # Set the density
    rho = 1.0

    # Set the number of blades in the turbine
    num_blades = 3

    # Width of Gaussian
    # Note: this sets the gaussian width to roughly twice the minimum cell length scale
    eps = problem.gaussian_width

    # initialize numpy torque
    rotor_torque_numpy_temp = 0.0

    # Blade length (turbine radius)
    L = problem.farm.radius[turb_i]

    #================================================================
    # Set Derived Constants
    #================================================================

    # Calculate the radial position of each actuator node
    rdim = np.linspace(0.0, L, problem.num_blade_segments)

    # Calculate width of an individual blade segment
    # w = rdim[1] - rdim[0]
    w = (rdim[1] - rdim[0])*np.ones(problem.num_blade_segments)
    w[0] = w[0]/2.0
    w[-1] = w[-1]/2.0

    # Calculate an array describing the x, y, z position of each actuator node
    # Note: The basic blade is oriented along the +y-axis
    blade_pos_base = np.vstack((np.zeros(problem.num_blade_segments),
                        rdim,
                        np.zeros(problem.num_blade_segments)))

    # Calculate the blade velocity
    angular_velocity = 2.0*np.pi*problem.rpm/60.0
    tip_speed = angular_velocity*L
    # tip_speed = 9.0

    # Specify the velocity vector at each actuator node
    # Note: A blade with span oriented along the +y-axis moves in the +z direction
    blade_vel_base = np.vstack((np.zeros(problem.num_blade_segments),
                           np.zeros(problem.num_blade_segments),
                           np.linspace(0.0, tip_speed, problem.num_blade_segments)))

    # Set the spacing pf each blade
    theta_vec = np.linspace(0.0, 2.0*np.pi, num_blades, endpoint = False)

    # Create unit vectors aligned with blade geometry
    # blade_unit_vec_base[:, 0] = points along rotor shaft
    # blade_unit_vec_base[:, 1] = points along blade span axis
    # blade_unit_vec_base[:, 2] = points tangential to blade span axis (generates a torque about rotor shaft)
    blade_unit_vec_base = np.array([[1.0, 0.0, 0.0],
                               [0.0, 1.0, 0.0],
                               [0.0, 0.0, 1.0]])

    #================================================================
    # Begin Calculating Turbine Forces
    #================================================================

    # Read cl and cd from the values specified in problem manager
    twist = np.array(problem.mtwist[turb_i], dtype = float)

    cl = np.array(problem.mcl[turb_i], dtype = float)
    cd = np.array(problem.mcd[turb_i], dtype = float)
    tip_loss = np.ones(problem.num_blade_segments)

    # Read the chord length from the values specified in the problem manager
    # c = L/20.0
    c = np.array(problem.mchord[turb_i], dtype = float)


    # Initialze arrays depending on what this function will be returning
    if dfd is None:
        tf_vec = np.zeros(np.size(problem.coords))
        tf_vec_for_power = np.zeros(np.size(problem.coords))
        lift_force = np.zeros((np.shape(problem.coords)[0], ndim))
        drag_force = np.zeros((np.shape(problem.coords)[0], ndim))

    elif dfd == 'c_lift':
        cl = np.ones(problem.num_blade_segments)
        dfd_c_lift = np.zeros((np.size(problem.coords), problem.num_blade_segments))

    elif dfd == 'c_drag':
        cd = np.ones(problem.num_blade_segments)
        dfd_c_drag = np.zeros((np.size(problem.coords), problem.num_blade_segments))

    elif dfd == 'chord':
        c = np.ones(problem.num_blade_segments)
        dfd_chord = np.zeros((np.size(problem.coords), problem.num_blade_segments))


    # Calculate the blade position based on current simTime and turbine RPM
    period = 60.0/problem.rpm
    # theta_offset = simTime/period*2.0*np.pi
    theta_offset = (simTime+0.5*dt)/period*2.0*np.pi
    # theta_offset = 0.0


    # Treat each blade separately
    for blade_ct, theta_0 in enumerate(theta_vec):
        # If the minimum distance between this mesh and the turbine is >2*RD,
        # don't need to account for this turbine
        if problem.min_dist[turb_i] > 2.0*(2.0*L):
            break

        theta = theta_0 + theta_offset

        # Generate a rotation matrix for this turbine blade
        Rx = rot_x(theta)
        Rz = rot_z(float(problem.farm.myaw[turb_i]))

        # Rotate the blade velocity in the global x, y, z, coordinate system
        # Note: blade_vel_base is negative since we seek the velocity of the fluid relative to a stationary blade
        # and blade_vel_base is defined based on the movement of the blade
        blade_vel = np.dot(Rz, np.dot(Rx, -blade_vel_base))

        # Rotate the blade unit vectors to be pointing in the rotated positions
        blade_unit_vec = np.dot(Rz, np.dot(Rx, blade_unit_vec_base))

        # Rotate the entire [x; y; z] matrix using this matrix, then shift to the hub location
        blade_pos = np.dot(Rz, np.dot(Rx, blade_pos_base))
        blade_pos[0, :] += problem.farm.x[turb_i]
        blade_pos[1, :] += problem.farm.y[turb_i]
        blade_pos[2, :] += problem.farm.z[turb_i]

        time_offset = 1
        if simTime_id < time_offset:
            theta_behind = theta_0 + 0.5*(problem.simTime_list[simTime_id]+simTime)/period*2.0*np.pi
        else:
            theta_behind = theta_0 + 0.5*(problem.simTime_list[simTime_id-time_offset]+simTime)/period*2.0*np.pi
            # if blade_ct == 0:
            #     print('SimTime = %f, using %f' % (simTime, problem.simTime_list[-time_offset]))

        Rx_alt = rot_x(theta_behind)
        blade_pos_alt = np.dot(Rz, np.dot(Rx_alt, blade_pos_base))
        blade_pos_alt[0, :] += problem.farm.x[turb_i]
        blade_pos_alt[1, :] += problem.farm.y[turb_i]
        blade_pos_alt[2, :] += problem.farm.z[turb_i]

        # Initialize space to hold the fluid velocity at each actuator node
        u_fluid = np.zeros((3, problem.num_blade_segments))

        # Generate the fluid velocity from the actual node locations in the flow
        for k in range(problem.num_blade_segments):

            u_fluid[:, k] = u_local(blade_pos_alt[0, k],
                     blade_pos_alt[1, k],
                     blade_pos_alt[2, k])

            u_fluid[:, k] -= np.dot(u_fluid[:, k], blade_unit_vec[:, 1])*blade_unit_vec[:, 1]

#         # Read the fluid velocity for this blade from mpi_u_fluid
#         start_pt = blade_ct*3*problem.num_blade_segments
#         end_pt = start_pt + 3*problem.num_blade_segments

#         u_fluid = mpi_u_fluid[turb_i, start_pt:end_pt]
#         u_fluid = np.reshape(u_fluid, (3, -1), 'F')

        for k in range(problem.num_blade_segments):
            u_fluid[:, k] -= np.dot(u_fluid[:, k], blade_unit_vec[:, 1])*blade_unit_vec[:, 1]


#         problem.blade_pos_previous[blade_ct] = blade_pos
                        
        # Form the total relative velocity vector (including velocity from rotating blade)
        u_rel = u_fluid + blade_vel
        
        u_rel_mag = np.linalg.norm(u_rel, axis=0)
        u_rel_mag[u_rel_mag < 1e-6] = 1e-6
        u_unit_vec = u_rel/u_rel_mag
        
        cl, cd, tip_loss = build_lift_and_drag(problem, u_rel, blade_unit_vec, rdim, twist, c)
                
        # Calculate the lift and drag forces using the relative velocity magnitude
        lift = tip_loss*(0.5*cl*rho*c*w*u_rel_mag**2)
        drag = tip_loss*(0.5*cd*rho*c*w*u_rel_mag**2)

        # Tile the blade coordinates for every mesh point, [numGridPts*ndim x problem.num_blade_segments]
        blade_pos_full = np.tile(blade_pos, (np.shape(problem.coords)[0], 1))

        # Subtract and square to get the dx^2 values in the x, y, and z directions
        dx_full = (problem.coordsLinear - blade_pos_full)**2

        # Add together to get |x^2 + y^2 + z^2|^2
        dist2 = dx_full[0::ndim] + dx_full[1::ndim] + dx_full[2::ndim]

        # Calculate the force magnitude at every mesh point due to every node [numGridPts x NumActuators]
        nodal_lift = lift*np.exp(-dist2/eps**2)/(eps**3 * np.pi**1.5)
        nodal_drag = drag*np.exp(-dist2/eps**2)/(eps**3 * np.pi**1.5)

        for k in range(problem.num_blade_segments):
            # The drag unit simply points opposite the relative velocity unit vector
            drag_unit_vec = -np.copy(u_unit_vec[:, k])
            
            # The lift is normal to the plane generated by the blade and relative velocity
            lift_unit_vec = np.cross(drag_unit_vec, blade_unit_vec[:, 1])

            # All force magnitudes get multiplied by the correctly-oriented unit vector
            vector_nodal_lift = np.outer(nodal_lift[:, k], lift_unit_vec)
            vector_nodal_drag = np.outer(nodal_drag[:, k], drag_unit_vec)

            if dfd == None:
                lift_force += vector_nodal_lift
                drag_force += vector_nodal_drag

            elif dfd == 'c_lift':
                for j in range(ndim):
                    dfd_c_lift[j::ndim, k] += vector_nodal_lift[:, j]

            elif dfd == 'c_drag':
                for j in range(ndim):
                    dfd_c_drag[j::ndim, k] += vector_nodal_drag[:, j]

            elif dfd == 'chord':
                for j in range(ndim):
                    dfd_chord[j::ndim, k] += vector_nodal_lift[:, j] + vector_nodal_drag[:, j]

            # Compute the total force vector [x, y, z] at a single actuator node
            actuator_lift = lift[k]*lift_unit_vec
            actuator_drag = drag[k]*drag_unit_vec

            # Note: since this will be used to define the force (torque) from fluid -> blade
            # we reverse the direction that otherwise gives the turbine force from blade -> fluid
            actuator_force = -(actuator_lift + actuator_drag)
            # actuator_force = -(actuator_lift - actuator_drag)

            # Find the component in the direction tangential to the blade
            tangential_actuator_force = np.dot(actuator_force, blade_unit_vec[:, 2])

            # Multiply by the distance away from the hub to get a torque
            actuator_torque = tangential_actuator_force*rdim[k]

            # Add to the total torque
            rotor_torque_numpy_temp += actuator_torque  ### Should this be an output?


    # Output the numpy version of rotor_torque
    problem.rotor_torque[turb_i] = rotor_torque_numpy_temp
    if rotor_torque_numpy_temp > 0:
        problem.rotor_torque_count[turb_i] = 1


    if dfd == None:
        # The total turbine force is the sum of lift and drag effects
        turbine_force = drag_force + lift_force
        turbine_force_for_power = -drag_force + lift_force

        # Riffle-shuffle the x-, y-, and z-column force components
        for k in range(ndim):
            tf_vec[k::ndim] = turbine_force[:, k]
            tf_vec_for_power[k::ndim] = turbine_force_for_power[:, k]

        # Remove near-zero values
        tf_vec[np.abs(tf_vec) < 1e-12] = 0.0

        # Add to the cumulative turbine force
        tf.vector()[:] += tf_vec

        # Create a cylindrical expression aligned with the position of this turbine
        cyld_expr = Expression(('sin(yaw)*(x[2]-zs)', '-cos(yaw)*(x[2]-zs)', '(x[1]-ys)*cos(yaw)-(x[0]-xs)*sin(yaw)'),
            degree=1,
            yaw=problem.farm.myaw[turb_i],
            xs=problem.farm.mx[turb_i],
            ys=problem.farm.my[turb_i],
            zs=problem.farm.z[turb_i])


        problem.cyld_expr_list[turb_i] = cyld_expr
        problem.cyld = cyld

    problem.first_call_to_alm = False

    tf.vector().update_ghost_values()

    if dfd == None:

        return tf

    elif dfd == 'c_lift':
        save_c_lift = False

        if save_c_lift:
            save_derivative_file(problem.params.folder+"timeSeries/",'dfdcl', dfd_c_lift)

        return dfd_c_lift

    elif dfd == 'c_drag':
        save_c_drag = False

        if save_c_drag:
            save_derivative_file(problem.params.folder+"timeSeries/",'dfdcd', dfd_c_drag)

        return dfd_c_drag

    elif dfd == 'chord':
        save_chord = False

        if save_chord:
            save_derivative_file(problem.params.folder+"timeSeries/",'dfdchord', dfd_chord)

        return dfd_chord


# WindSE Duplication and Setup
The next code block is one way to create the WindSE objects expected by the actuator line function in a very direct manner.  Normally these objects, values, nested objects, etc., are set with calls to the relevant ...Manager.py file, but here we set them directly to make it clearer what the inputs and outputs to the actuator line function are. This is the bare minimum list and not representative of all the items stored in these objects, but it should be adequate for calculating actuator line forces.

In [2]:
# Create a 3D box mesh from (-100, -100, 0) to (100, 100, 200)
# with RES x RES x RES total nodes
res = 30

mesh = BoxMesh(Point(-100, -100, 0),
              Point(100, 100, 200),
              res, res, res)

# Create a vector function space (used by fluid velocity and turbine force)
V = VectorFunctionSpace(mesh, 'P', 2)


# ================================================================
# Create a problem object (normally from ProblemManager) and the nested objects it inherits
# that represent what a WindSE run would normally look like at the point of calling
# the actuator line functions
# ================================================================
class BlankObject:
    pass

problem = BlankObject()


# ================================================================
# dom, from DomainManager
# ================================================================
problem.dom = BlankObject()
# ndim, the dimension of the mesh, either 2 or 3
problem.dom.dim = mesh.geometry().dim()


# ================================================================
# farm, from WindFarmManager
# ================================================================
problem.farm = BlankObject()
# radius, a list with num_turbs elements, radius[0] = blade radius of turbine 0
problem.farm.radius = [65.0]
# myaw, a list with num_turbs elements, myaw[0] = yaw angle of turbine 0
problem.farm.myaw = [0]
# mx, a list with num_turbs elements, mx[0] = x-position turbine 0
problem.farm.mx = [0]
problem.farm.x = [0]
# my, a list with num_turbs elements, my[0] = y-position turbine 0
problem.farm.my = [0]
problem.farm.y = [0]
# z, a list with num_turbs elements, z[0] = z-position turbine 0 (hub height, if ground level = 0)
problem.farm.z = [110.0]


# ================================================================
# fs, from FunctionSpaceManager
# ================================================================
problem.fs = BlankObject()
# V, a vector function space associated with the mesh
problem.fs.V = V


# ================================================================
# Variable set directly in ProblemManager
# ================================================================
# gaussian_width, the characteristic size of the gaussian, big enough to ensure it intersects mesh points,
# small enough to ensure a relatively "sharp" application of the blade force
hmin = mesh.hmin()/np.sqrt(3.0)
problem.gaussian_width = 3.0*hmin
# num_blade_segments, the number of actuator nodes (separate gaussian distributions), used to represent 
# the blade from root to tip
problem.num_blade_segments = 10

def build_lift_and_drag_tables(airfoil_data_path):

    # Determine the number of files in airfoil_data_path
    num_files = len(glob.glob('%s/*.txt' % (airfoil_data_path)))

    for file_id in range(num_files):
        # print('Reading Airfoil Data #%d' % (file_id))
        data = np.genfromtxt('%s/af_station_%d.txt' % (airfoil_data_path, file_id), skip_header=1, delimiter=' ')

        if file_id == 0:
            # If this is the first file, store the angle data
            interp_angles = data[:, 0]
            num_angles = np.size(interp_angles)

            # If this is the first file, allocate space for the tables        
            lift_table = np.zeros((num_angles, num_files))
            drag_table = np.zeros((num_angles, num_files))

        # Store all the lift and drag data in the file_id column
        lift_table[:, file_id] = data[:, 1]
        drag_table[:, file_id] = data[:, 2]

    return lift_table, drag_table, interp_angles

# Get the lift and drag values that will later be used in interpolation
lift_table, drag_table, interp_angles = build_lift_and_drag_tables('airfoil_polars')
problem.lift_table = lift_table
problem.drag_table = drag_table
problem.interp_angles = interp_angles

# Get the chord, twist, cL, and cD values from IEA RWT data
actual_turbine_data = np.genfromtxt('Input_Data/baseline.csv', delimiter = ',', skip_header = 1)

actual_x = actual_turbine_data[:, 0]
actual_chord = actual_turbine_data[:, 1]
actual_twist = actual_turbine_data[:, 2]/180.0*np.pi
actual_cl = actual_turbine_data[:, 3]
actual_cd = actual_turbine_data[:, 4]

chord_interp = interp.interp1d(actual_x, actual_chord)
twist_interp = interp.interp1d(actual_x, actual_twist)
cl_interp = interp.interp1d(actual_x, actual_cl)
cd_interp = interp.interp1d(actual_x, actual_cd)

interp_points = np.linspace(0.0, 1.0, problem.num_blade_segments)

# Generate the interpolated values
chord = chord_interp(interp_points)
twist = twist_interp(interp_points)
cl = cl_interp(interp_points)
cd = cd_interp(interp_points)


# mtwist, the twist of the sections along the blade length
problem.mtwist = [twist]
# mcl, the coefficient of lift along the blade
problem.mcl = [cl]
# mcl, the coefficient of drag along the blade
problem.mcd = [cd]
# mcl, the chord length along the blade
problem.mchord = [chord]

# rpm, the rotational speed of the blade
problem.rpm = 10.0
# The coordinates at each point of the mesh, indexed using the same ordering as the vector function space
coords = problem.fs.V.tabulate_dof_coordinates()
coords = np.copy(coords[0::problem.dom.dim, :])
problem.coords = coords
problem.coordsLinear = np.copy(coords.reshape(-1, 1))
# min_dist, a cutoff distance, if the min_distance < threshold, this section of mesh is close enough to
# the current turbine that the forcing effect is non-negligible and must be counted
problem.min_dist = [0.0]
# rotor_torque, a list with num_turbs elements that holds the rotor torque, each gaussian contributes a bit of force
problem.rotor_torque = [0]
problem.rotor_torque_count = [0]
# cyld_expr_list, a list with num_turbs elements where each entry is a cylinder 
# encapsulating the turbine (used for later calculting the power) 
problem.cyld_expr_list = [0]


Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.


# Run the function
The last cell runs calls the actuator line function using the problem definition above for a number of timesteps, intended to duplicate how the function would be updated during an actual windSE solve.  The `u_local` variable (the ambient flow) doesn't change during the iteration as it would during a real solve, only the position of the blades and the velocity induced by that rotation is counted.

In [3]:
# ================================================================
# Variables passed in directly as part of looping through all turbines and timesteps
# ================================================================

# Create a dummy fluid velocity field
u_local = Function(problem.fs.V)

# Set the values of the dummy velocity field to be a linear shear in the +x direction
inflow_profile = Expression(('9.0', '0', '0'), degree=2)
u_local.interpolate(inflow_profile)

# Currently, only a single turbine
turb_i = 0

# Not currently being used, but left in for easily merging your notebook changes to WindSE
mpi_u_fluid = 1

# The number of timesteps to simulate
tSteps = 10

# The timestep size
dt = 0.1

# The accumulated simulation time
simTime = 0.0

# simTime_list, a list containing all simulation timesteps accumulated
problem.simTime_list = []

# Initialize file pointers for writing
fp_vel = File('output/velocity.pvd')
fp_tf = File('output/turbine_force.pvd')

# ================================================================
# Call the actuator line function and return the turbine force being projected on the computational grid
# ================================================================

for k in range(tSteps):
    print('Timestep %d of %d, t = %.2f' % (k+1, tSteps, simTime))
    
    # Append the simTime list and set a convenience variable
    problem.simTime_list.append(simTime)
    simTime_id = k
    
    tf = UpdateActuatorLineForce(problem, u_local, simTime_id, dt, turb_i, mpi_u_fluid, dfd=None, verbose=False)
    
    # Increment the total simTime
    simTime += dt
    
    # Save the velocity and turbine force for visualization in ParaView (a debugging exercise only)
    u_local.rename('velocity', 'velocity')
    fp_vel << (u_local, k)

    tf.rename('turbine_force', 'turbine_force')
    fp_tf << (tf, k)


Timestep 1 of 10, t = 0.00
Timestep 2 of 10, t = 0.10
Timestep 3 of 10, t = 0.20
Timestep 4 of 10, t = 0.30
Timestep 5 of 10, t = 0.40
Timestep 6 of 10, t = 0.50
Timestep 7 of 10, t = 0.60
Timestep 8 of 10, t = 0.70
Timestep 9 of 10, t = 0.80
Timestep 10 of 10, t = 0.90
